In [ ]:
import skimage as sk
from skimage.io import imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
from cellpose.io import imread
from cellpose import io, utils
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_measurements(mask,props):
    measurements = sk.measure.regionprops_table(mask,properties=props)
    df = pd.DataFrame.from_dict(measurements)
    return df

In [ ]:
def save(save_path, img_name, masks, merged_df):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
        outlines_path = os.mkdir(os.path.join(save_path,'ImageJ_ROIs'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        dataframe_path = os.path.join(save_path,'measurements')
        outlines_path = os.path.join(save_path,'ImageJ_ROIs') 
    imsave(os.path.join(masks_path,'masks_'+img_name[:-4]+'.tif'),masks)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))
    io.save_rois(masks,img_name)

In [ ]:
all_files = sorted(glob('E:/Lien_Lab/Sam/OIC-130_CellCountScript/All_Imgs/Imgs/*.tif'))
all_img = [imread(file) for file in all_files]

In [ ]:
test_img = all_img[0]

In [ ]:
io.logger_setup()
model = models.CellposeModel(gpu=True,pretrained_model='IncucyteV4')
# model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
#masks, _, _= model.eval(test_img, diameter=20, channels=channels)

In [ ]:
print(np.max(masks))

In [ ]:
props = ['label','area']
save_path = 'E:/Lien_Lab/Sam/OIC-130_CellCountScript/'
for i in range(len(all_img)):
    img_name = os.path.basename(all_files[i])
    masks, _, _ = model.eval(all_img[i], diameter=20, channels=channels)
    df = get_measurements(masks,props)
    save(save_path=save_path, img_name=img_name, masks=masks, merged_df=df)

In [ ]:
from cellpose import io, utils

# image_name is file name of image
# masks is numpy array of masks for image
base = os.path.splitext(image_name)[0]
outlines = utils.outlines_list(masks)
io.outlines_to_text(base, outlines)